In [1]:
import sqlite3
import pandas as pd 
from tqdm.auto import tqdm
from IPython.display import display, HTML
import re
import datetime
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json 

# Visualize HTML Data

In [226]:
# ## leave the same
if os.path.exists('../data/tr-story'):
    dp_path = '../data/sheena-sample/sample-nyt-processed.pkl'
else:
    dp_path = 'sample-nyt-processed.pkl'
processed_diffs_df = pd.read_pickle(dp_path)

In [234]:
processed_diffs_df.head(1)

,source,a_id,version_old,version_new,created_old,created_new,archive_url_old,archive_url_new,title_comp_html,article_comp_html
0,nyt,547988,0,1,2012-08-26 22:55:00+00:00,2012-08-27 02:55:00+00:00,https://www.newssniffer.co.uk/articles/547988/...,https://www.newssniffer.co.uk/articles/547988/...,<h3>Headline</h3>\n ...,<table>\n<tr><th>s id</th><th>Old Version</th>...


In [238]:
processed_diffs_df['a_id'].iloc[0]

'547988'

In [242]:
### ---- Change as Needed ----
idx = 12 ## increment this

## leave this the same
source, a_id, version_old, version_new, created_old, created_new, url_old, url_new, title_html, article_html = (
    processed_diffs_df.loc[lambda df: df['a_id'] == '554965']
#                       .loc[lambda df: df['version_old'] == '0']
#                       .loc[lambda df: df['version_new'] == '1']
                      .iloc[0]
)

print('source: %s' % source)
print('article id: %s' % a_id)
print('old version num: %s' % version_old)
print('new version num: %s' % version_new)
print('old date created: %s' % created_old)
print('new date created: %s' % created_new)

display(HTML(title_html))
display(HTML('<h3>Article Body</h3>'))
display(HTML(article_html))

print()
print('old url: %s' % url_old)
print('new url: %s' % url_new)

source: nyt
article id: 554965
old version num: 1
new version num: 2
old date created: 2012-09-15 13:30:00+00:00
new date created: 2012-09-15 16:25:00+00:00


Old Version,New Version
Anti - Putin Protesters March in Moscow,"Undaunted by Arrests and Sanctions , Opposition Marches Against Putin"


s id,Old Version,New Version
0,"MOSCOW—Russia’s opposition movement took to the streets on Saturday for another anti-Putin demonstration, hoping to show that its ranks were undaunted by a battery of government restrictions or the two-year prison sentences handed down last month to members of the punk rock band Pussy Riot.",
1,"In the three months since the last demonstration, the government has taken several steps to rein in street protests with measures that are far more punitive than it has embraced since Vladimir V. Putin first became president.",
2,"The most notable is the use of criminal courts and investigations as a lever against members of the political opposition, a step that Mr. Putin had avoided for many years.",
3,There was an intense police presence on Saturday and a sense of risk hung over the march.,
4,"The police estimated the size of the crowd at 14,000, though organizers put the number far higher, in the tens of thousands.",
5,,"This protest, like the ones that first jolted the Kremlin nine months ago, featured mockery of President Vladimir V. Putin — largely sendups of a recent stunt in which Mr. Putin flew in a hang glider at the head of a flock of cranes."
6,,"The trademark neon balaclavas of Pussy Riot, whose members were prosecuted for singing an anti-Putin song in a cathedral, appeared on buttons and balloons and, in a basket carried by one marcher, on three disembodied mannequin heads."
7,,"The police estimated the crowd at 14,000, roughly the same figure they offered after a similar protest in June."
8,,Organizers’ estimates were much larger.
9,,"No arrests were reported, and there was no repeat of chaotic clashes with the police that occurred at a protest in May."



old url: https://www.newssniffer.co.uk/articles/554965/versions/970445
new url: https://www.newssniffer.co.uk/articles/554965/versions/970501


# Process data

In [221]:
# merged_diffs_df
# old_archive_url	new_archive_url	

In [225]:
import sys
sys.path.append('../util')
import util_newssniffer_parsing as unp

with open('/Users/alex/Downloads/results-20210407-000919.json') as f:
    df = pd.DataFrame(json.load(f))

merged_diffs_df = (df
 .assign(text_old=
   lambda df: df.apply(lambda x: list(map(lambda y: {'text': y[0], 'tag': y[1]}, zip(x['sents_old'], x['tags_old']))), axis=1))
 .assign(text_new=
   lambda df: df.apply(lambda x: list(map(lambda y: {'text': y[0], 'tag': y[1]}, zip(x['sents_new'], x['tags_new']))), axis=1))
.drop(['sents_old', 'sents_new', 'tags_old', 'tags_new'], axis=1)
)

def fix_vars(v):
    '''For some weird reason, sometimes lines without text still have a +/- tag.'''
    for v_i in v:
        if v_i['text'] == '':
            v_i['tag'] = ' '

for t in ['old_headline', 'new_headline']:
    merged_diffs_df[t] = merged_diffs_df[t].fillna('')

html_comps = []
for idx in tqdm(range(len(merged_diffs_df))):
    source, a_id, version_old, version_new, title_old, title_new, created_old, created_new, old_archive_url, new_archive_url, vars_old, vars_new = (
        merged_diffs_df.iloc[idx]
    )
    fix_vars(vars_old), fix_vars(vars_new)
    vars_title_old, vars_title_new = unp.get_word_diffs(title_old, title_new)
    title_html1, title_html2 = unp.html_compare_sentences(vars_title_old, vars_title_new)

    output = {
        'source': source,
        'a_id': a_id,
        'version_old' : version_old ,
        'version_new' : version_new ,
        'created_old': created_old,
        'created_new': created_new,
        'archive_url_old': old_archive_url, 
        'archive_url_new': new_archive_url,
        'title_comp_html': '''<h3>Headline</h3>
                              <table>
                                  <tr><th>Old Version</th><th>New Version</th></tr>
                                  <tr><td>%s</td><td>%s</td></tr>
                              </table>
        ''' % (title_html1, title_html2),
        'article_comp_html': unp.html_compare_articles(vars_old, vars_new, use_sids=True),
    }
    html_comps.append(output)

html_df = (pd.DataFrame(html_comps)
 .sort_values(['a_id', 'version_old', 'version_new'])
 .loc[lambda df: df['source'] == 'nyt']
#  .iloc[2:]
 .reset_index(drop=True)
 .to_pickle('../data/sheena-sample/sample-nyt-processed.pkl')
)

  0%|          | 0/500 [00:00<?, ?it/s]

In [137]:
# bugs in WP (so far, just WP) that need to be fixed:
# sometimes the versions are not in the right order (in the case we saw, the datetime was not the same)
    ## source: wp
    ## article id: 1066322
    ## old version num: 0, new version num: 1
    ## old version num: 1, new version num: 2
    
# sometimes the first sentence gets moved very far down.
    ## source: wp
    ## article id: 1070540
    ## old version num: 0, new version num: 1
    ## source: wp
    ## article id: 1070931
    ## old version num: 2, new version num: 3
    ## source: wp
    ## article id: 1071219
    ## old version num: 0, new version num: 1
    ## source: nyt
    ## article id: 554965
    ## old version num: 1, new version num: 2
    
# when there are sentences that are dashes they force a lot of other sentences to get compressed
    ## source: wp
    ## article id: 1070624
    ## old version num: 5, new version num: 6
    ## source: wp
    ## article id: 1071254
    ## old version num: 2, new version num: 3
    
con = sqlite3.connect('../data/diffengine-diffs/db/newssniffer-washpo.db')
entry_version = pd.read_sql('select * from entryversion', con=con)
entry_version.loc[lambda df: df['entry_id'] == 1070931].loc[lambda df: df['version'] == 3]['summary'].iloc[0]

# Audit

In [243]:
wp_sentence_diff_con = sqlite3.connect('../data/diffengine-diffs/output/wp.db')

In [245]:
wp_sentence_con = sqlite3.connect('../data/diffengine-diffs/db/newssniffer-washpo.db')

In [244]:
pd.read_sql('select * from sentence_diffs limit 5', con=wp_sentence_diff_con)

,a_id,tag_old,source,version_new,version_old,s_idx,tag_new,sent_new,sent_old
0,1021717,,wp,1,0,0,+,"ROSEBURG, Ore.—A rural community college becam...",
1,1021717,,wp,1,0,1,+,"Officials said nine people were killed, plus t...",
2,1021717,,wp,1,0,10,+,Get out of there!”,
3,1021717,,wp,1,0,100,+,"McCrae Kittelman, 17, was in the Educational S...",
4,1021717,,wp,1,0,101,+,It was 10:40 a.m.,


In [277]:
articles = pd.read_sql('''
    SELECT entry_id, version, summary FROM entryversion
''', con=wp_sentence_con)

In [278]:
old_versions = pd.read_sql('''
    SELECT a_id, version_old, GROUP_CONCAT(sent_old, ' ') as full_story
    FROM (
        SELECT a_id, version_old, sent_old
        FROM sentence_diffs 
        ORDER BY a_id, version_old, s_idx ASC
    )
    GROUP BY a_id, version_old
''', con=wp_sentence_diff_con)



In [376]:
new_versions = pd.read_sql('''
    SELECT a_id, version_new, GROUP_CONCAT(sent_new, '<BREAK>') as sentences
    FROM (
        SELECT a_id, version_new, sent_new
        FROM sentence_diffs 
        ORDER BY a_id, version_new, s_idx ASC
    )
    GROUP BY a_id, version_new
''', con=wp_sentence_diff_con)

In [377]:
new_versions['sentences'] = new_versions['sentences'].str.split('\<BREAK\>')

In [461]:
def split_sents(s):
    if '.' in s:
        doc = nlp(s)
        return list(map(lambda x: x.text, doc.sents))
    return [s]
        
    
def get_index(s, b):
    if s in b:
        idx = b.index(s)
    else:
        matches = find_near_matches(s, b, max_l_dist=5)
        if len(matches) == 1:
            idx = matches[0].start
        else:
            return -1
    return idx
    

def order_sentences(sentences, base_text):
    output = []
    sentences = list(filter(lambda x: x != '', sentences))
    for s in sentences:
        if len(sentence) > 2:
            idx = get_index(s, base_text)
            if idx == -1:
                for split_s in split_sents(s):
                    output.append({
                        'idx': get_index(split_s, base_text),
                        'sentence': split_s
                    })
            else:
                output.append({
                    'idx': idx,
                    'sentence': s
                })
    return output

In [410]:
comparison = (new_versions
 .merge(articles, left_on=['a_id', 'version_new'], right_on=['entry_id', 'version'])
 .assign(summary=lambda df: df['summary'].str.replace('</p><p>',' '))
)

In [386]:
from fuzzysearch import find_near_matches

In [535]:
outputs = []
for idx, sent, full in tqdm(comparison[['sentences', 'summary']].head(5000).itertuples(), total=len(comparison)):
    output = order_sentences(sent, full)
    outputs.append(output)

  0%|          | 0/48644 [00:00<?, ?it/s]

In [536]:
stats = []
for o in tqdm(outputs):
    output_dict = {}
    t_df = pd.DataFrame(o)

    output_dict['unfindable_sent'] = (t_df['idx'] == -1).any()
    output_dict['inverted'] = (t_df['idx']
     .pipe(lambda s: pd.Series(s[:-1].values - s[1:].values))
     .pipe(lambda s: s >0)
     .any()
    )
    stats.append(output_dict)

In [547]:
full_stats_df = pd.concat([
    pd.Series(outputs).to_frame('sents'),
    pd.DataFrame(stats),
    comparison[['sentences', 'summary']].head(5000)
], axis=1)

In [545]:
stats_df['unfindable_sent'].mean()

0.0174

In [544]:
stats_df['inverted'].mean()

0.4764

In [594]:
t = (
    full_stats_df
     .loc[lambda df: df['inverted'] == True]
     .iloc[3]
)

In [601]:
import jellyfish
def check_resort(t):
    t_df = pd.DataFrame(t['sents'])
    return jellyfish.levenshtein_distance(
        ' '.join(list(t_df.sort_values('idx')['sentence'])),
        ' '.join(t['summary'].split())
    )

In [607]:
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas()

levs = []
for idx, r in tqdm((full_stats_df
 .loc[lambda df: df['inverted'] == True]
 .loc[lambda df: df['unfindable_sent'] == False]).iterrows(), total=len(full_stats_df)):
    levs.append(check_resort(r))

In [621]:
levs

[0,
 5,
 5,
 5,
 0,
 15,
 15,
 15,
 14,
 14,
 14,
 14,
 16,
 16,
 16,
 8,
 24,
 24,
 24,
 12,
 12,
 12,
 10,
 10,
 10,
 12,
 200,
 200,
 200,
 198,
 198,
 198,
 198,
 16,
 8,
 174,
 195,
 6,
 6,
 10,
 10,
 266,
 2,
 8,
 18,
 173,
 151,
 173,
 215,
 18,
 18,
 9,
 2,
 12,
 351,
 351,
 160,
 159,
 163,
 163,
 163,
 163,
 179,
 49,
 6,
 6,
 6,
 6,
 6,
 6,
 4,
 6,
 4,
 6,
 30,
 108,
 108,
 108,
 6,
 6,
 5,
 104,
 60,
 103,
 121,
 263,
 253,
 253,
 154,
 158,
 424,
 424,
 44,
 44,
 391,
 287,
 2,
 148]

In [622]:
r

sents              [{'idx': 0, 'sentence': 'RICHMOND—Virginia Att...
unfindable_sent                                                False
inverted                                                        True
sentences          [RICHMOND—Virginia Attorney General Mark R. He...
summary            RICHMOND — Virginia Attorney General Mark R. H...
Name: 233, dtype: object

In [623]:
t_df = pd.DataFrame(r['sents'])

In [625]:
import util.util_diffengine_parsing as p

In [628]:
d = p.diffHandler()

In [631]:
display(HTML(d.list_to_html(d.rolling_window(d.output_diff_lists(
    ' '.join(r['summary'].split()), 
    ' '.join(list(t_df.sort_values('idx')['sentence']))
)))))

In [533]:
(stats_df
 .loc[lambda df: df['unfindable_sent']]
 .assign(non_sents=lambda df: df['sents'].apply(lambda x: list(filter(lambda y: y['idx'] == -1, x))))
 ['non_sents']
#  .str.len()
 .iloc[4]
 [0]['sentence']
#  .loc[lambda df: df['idx'] == -1]['sentence']
)

'A crowd-sourced count on Reddit of any gun incident in which four or more were shot The newspaper’s tally, including incidents in which four or more people were killed, was 146 mass shootings since 2006.'

In [452]:
sent
for sentence in sent:
    if len(sentence) > 2:
        if sentence in full:
            idx = full.index(sentence)
        else:
            matches = find_near_matches(sentence, full, max_l_dist=5)
            if len(matches) == 1:
                idx = matches[0].start
            else:
                break

In [453]:
sentence

'Peyton Manning Manning, still in his football pants, gestured as he described one of his 32 completions in 43 attempts; the things he saw as he passed for 400 yards and two touchdowns. “I was truly taking things slowly,” Manning said, “kind of phase by phase.”'

In [454]:
full

'DENVER —  They gathered at the two locker stalls near the door, two brothers stopping to congratulate and hug Archie and Olivia Manning’s middle boy. Cooper Manning and youngest brother Eli, the New York Giants quarterback, stood as Peyton sat and removed his cleats. They were smiling, and before long, cameras began clicking to capture the moment. “You definitely have to take time to savor the moment,” Peyton Manning   said a few minutes later. “I know I will.” Manning had led the Denver Broncos to the AFC championship, a 26-16 win against the New England Patriots and rival quarterback Tom Brady, but more than that Sunday’s win was the final turn of a stunning comeback. Manning, 37, is returning to the Super Bowl, and below the back of the gray AFC championship hat, a scar was visible on his neck — a reminder of the injury that nearly ended his career. “I had some doubts,” Eli said. “I had some worries for sure.” Two years ago, Manning missed the 2011 season with a herniated disk in h

In [424]:
full

'On a recent Friday evening in Kabul, I gathered with friends at the Lebanese restaurant that had long been a convivial and secure oasis in a harsh and unpredictable country. The occasion was a farewell meal before I left for the States — a cherished ritual in my many visits to Afghanistan over the past decade. As always, after an abundant assortment of mezze, we raised our discreet cups of “white tea” and promised to meet there next time. As always, the proprietor, Kamel Hamade, a dapper businessman from Beirut, refused to let us pay. “Give the money to help the animals instead,” he would insist. Kamel was an animal lover, political gadfly and the genial host of La Taverna du Liban, a cozy bistro that catered to the foreign and local elite — Western aid workers, Middle Eastern entrepreneurs, Afghan ministry officials and foreign correspondents like me. There was a frisson of intrigue amid the hookah smoke and hushed chatter. We jokingly called it the “Rick’s Café” of Kabul. There was 

In [404]:
comparison.apply(lambda x: order_sentences(x['sentences'], x['summary']), axis=1)

At 7 p.m., the busiest dinner hour of the week, a Taliban suicide bomber detonated his explosives outside the front gate, next to Jeff’s pen, and two gunmen shot their way inside, raking the dining room with gunfire. And Jeff vanished.
[]
Now Kiev is trying to be more like Moscow in legal ways, as well — enacting Friday a set of draconian laws that would prohibit almost any protest, curtail freedom of speech, hobble the press, enable the government to ban citizens from using the Internet and classify advocacy groups as “foreign agents” if they receive money from abroad.
[]
DENVER—They gathered at the two locker stalls near the door, two brothers stopping to congratulate and hug Archie and Olivia Manning’s middle boy.
[]
Peyton Manning Manning, still in his football pants, gestured as he described one of his 32 completions in 43 attempts; the things he saw as he passed for 400 yards and two touchdowns. “I was truly taking things slowly,” Manning said, “kind of phase by phase.”
[]
DENVER

UnboundLocalError: local variable 'idx' referenced before assignment

In [283]:
articles.head(1)

,entry_id,version,summary
0,728205,0,MOSCOW — Russians love to think of their neigh...


In [289]:
from tqdm.auto import tqdm

In [297]:
tqdm.pandas()

new_sims = (new_versions
 .merge(articles, left_on=['a_id', 'version_new'], right_on=['entry_id', 'version'])
 .assign(full_story=lambda df: df['full_story'].str.replace('</p><p>',' '))
 .head(10000)
 .progress_apply(lambda x: SequenceMatcher(None, x['full_story'], x['summary']).ratio(), axis=1)
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [319]:
sim_df = pd.concat([
    new_versions
     .merge(articles, left_on=['a_id', 'version_new'], right_on=['entry_id', 'version'])
     .assign(full_story=lambda df: df['full_story'].str.replace('</p><p>',' '))
     .head(10000),
    new_sims.to_frame('similarity')
], axis=1)

In [356]:
t_new = sim_df.loc[lambda df: df['similarity'] > .9].loc[9995]['full_story']

In [357]:
t_old = sim_df.loc[lambda df: df['similarity'] > .9].loc[9995]['summary']

In [327]:
import difflib

In [336]:
from nltk.tokenize import sent_tokenize

In [354]:
difflib.SequenceMatcher(None, unp.split_sents(t1), unp.split_sents(t2)).ratio()

0.918918918918919

In [349]:
from IPython.display import display, HTML

In [350]:
import sys
sys.path.append('../')

In [351]:
import util_newssniffer_parsing as unp

In [352]:
changes_old_sent, changes_new_sent = unp.get_list_diff(unp.split_sents(t1), unp.split_sents(t2))

In [353]:
display(HTML(unp.html_compare_articles(changes_old_sent, changes_new_sent)))

Old Version,New Version
"DALLAS—Ebola patient Thomas Eric Duncan told his fiancee the day he was diagnosed last week that he regrets exposing her to the deadly virus and had he known he was carrying Ebola, he would have “preferred to stay in Liberia and died than bring this to you,” a family friend said.","DALLAS—Ebola patient Thomas Eric Duncan told his fiancee the day he was diagnosed last week that he regrets exposing her to the deadly virus and had he known he was carrying Ebola, he would have “preferred to stay in Liberia and died than bring this to you,” a family friend said."
"“ He told her , ‘ I ’m so sorry all of this is happening . . . .","“ He told her , ‘ I ’m so sorry all of this is happening . . . ."
I would not put the love of my life in danger.’ ”,I would not put the love of my life in danger.’ ”
Family members gathered in Dallas and were able to see Duncan on Monday at Texas Health Presbyterian Hospital via a laptop camera in his hospital room.,Family members gathered in Dallas and were able to see Duncan on Monday at Texas Health Presbyterian Hospital via a laptop camera in his hospital room.
“He is not responsive at all.,“He is not responsive at all.
"They said he was drugged and they put him in deep sleep,” said Lloyd, who was with the family.","They said he was drugged and they put him in deep sleep,” said Lloyd, who was with the family."
She said the family members were saddened by the sight of Duncan.,She said the family members were saddened by the sight of Duncan.
"“The mother, this was the first time seeing her son since he got to the States.","“The mother, this was the first time seeing her son since he got to the States."
"She has not seen him for 12 years, and the first time she saw him was through a monitor,” said Lloyd.","She has not seen him for 12 years, and the first time she saw him was through a monitor,” said Lloyd."
"“She was very, very emotional.","“She was very, very emotional."


In [355]:
t1

'DALLAS—Ebola patient Thomas Eric Duncan told his fiancee the day he was diagnosed last week that he regrets exposing her to the deadly virus and had he known he was carrying Ebola, he would have “preferred to stay in Liberia and died than bring this to you,” a family friend said. “He told her, ‘I’m so sorry all of this is happening. . . . I would not put the love of my life in danger.’ ” Family members gathered in Dallas and were able to see Duncan on Monday at Texas Health Presbyterian Hospital via a laptop camera in his hospital room. “He is not responsive at all. They said he was drugged and they put him in deep sleep,” said Lloyd, who was with the family. She said the family members were saddened by the sight of Duncan. “The mother, this was the first time seeing her son since he got to the States. She has not seen him for 12 years, and the first time she saw him was through a monitor,” said Lloyd. “She was very, very emotional. She told him to be strong, that she is praying for h

In [274]:
from difflib import SequenceMatcher